In [1]:
import openai
import os

In [2]:
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
# Generating the prompt for the program
def create_test_prompt(topic,num_questions,num_possible_answers):
    prompt = f"Create a multiple choice quiz on the topic of {topic} consisting of {num_questions} questions. "\
    + f"Each question should have {num_possible_answers} options. "\
    + f"Also include the correct answer for each question using the starting phrase 'Correct Answer:' "
    return prompt


In [4]:
# getting the response from the OpenAI
response = openai.Completion.create(
    engine = 'text-davinci-003',
    prompt = create_test_prompt('US History', 4, 4),
    max_tokens = 256,
    temperature = 0.7
)

In [5]:
print(response['choices'][0]['text'])



Q1. Who was the first president of the United States?
A. Thomas Jefferson
B. George Washington
C. John Adams
D. Abraham Lincoln
Correct Answer: B. George Washington

Q2. What event is considered to be the start of World War I?
A. The bombing of Pearl Harbor
B. The assassination of Archduke Franz Ferdinand
C. The signing of the Treaty of Versailles
D. The invasion of Normandy
Correct Answer: B. The assassination of Archduke Franz Ferdinand

Q3. When did the United States become an independent nation?
A. 1776
B. 1787
C. 1789
D. 1798
Correct Answer: A. 1776

Q4. When was the Emancipation Proclamation issued?
A. 1863
B. 1865
C. 1867
D. 1868
Correct Answer: A. 1863


In [6]:
# Since the openai is generating the whole question and answers
# We need to create a function for Student View
def create_student_view(test, num_questions):
    student_view = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if not line.startswith("Correct Answer:"):
            student_view[question_number] += line + '\n'
        else:
            if question_number < num_questions:
                question_number += 1
                student_view[question_number] = ''
    return student_view

In [7]:
result = create_student_view(response['choices'][0]['text'],4)

In [8]:
for key in result:
    print(result[key])



Q1. Who was the first president of the United States?
A. Thomas Jefferson
B. George Washington
C. John Adams
D. Abraham Lincoln


Q2. What event is considered to be the start of World War I?
A. The bombing of Pearl Harbor
B. The assassination of Archduke Franz Ferdinand
C. The signing of the Treaty of Versailles
D. The invasion of Normandy


Q3. When did the United States become an independent nation?
A. 1776
B. 1787
C. 1789
D. 1798


Q4. When was the Emancipation Proclamation issued?
A. 1863
B. 1865
C. 1867
D. 1868



In [9]:
# Then we will need to extract the answer so that we can make our comparison
def extract_answer(test, num_questions):
    answers = {1:''}
    question_number = 1
    for line in test.split("\n"):
        if line.startswith("Correct Answer:"):
            answers[question_number] += line + '\n'
        
            if question_number < num_questions:
                question_number += 1
                answers[question_number] = ''
    return answers

In [10]:
result2 = extract_answer(response['choices'][0]['text'], 4)

In [11]:
for key in result2:
    print(result2[key])

Correct Answer: B. George Washington

Correct Answer: B. The assassination of Archduke Franz Ferdinand

Correct Answer: A. 1776

Correct Answer: A. 1863



In [12]:
student_view = create_student_view(response['choices'][0]['text'], 4)

In [13]:
answers = extract_answer(response['choices'][0]['text'], 4)

In [14]:
# Take the Exam

def take(student_view):
    student_answers = {}
    for question, question_view in student_view.items():
        print(question_view)
        answer = input("Enter your answer: ")
        student_answers[question] = answer
    return student_answers

In [15]:
# student_view.items()

In [16]:
student_answers = take(student_view)



Q1. Who was the first president of the United States?
A. Thomas Jefferson
B. George Washington
C. John Adams
D. Abraham Lincoln

Enter your answer: a

Q2. What event is considered to be the start of World War I?
A. The bombing of Pearl Harbor
B. The assassination of Archduke Franz Ferdinand
C. The signing of the Treaty of Versailles
D. The invasion of Normandy

Enter your answer: s

Q3. When did the United States become an independent nation?
A. 1776
B. 1787
C. 1789
D. 1798

Enter your answer: d

Q4. When was the Emancipation Proclamation issued?
A. 1863
B. 1865
C. 1867
D. 1868

Enter your answer: a


In [17]:
answers

{1: 'Correct Answer: B. George Washington\n',
 2: 'Correct Answer: B. The assassination of Archduke Franz Ferdinand\n',
 3: 'Correct Answer: A. 1776\n',
 4: 'Correct Answer: A. 1863\n'}

In [18]:
# Comparing the actual result and the student result
def grade(correct_answer_dict, student_answers):
    correct_answers = 0
    for question, answer in student_answers.items():
        if answer.upper() == correct_answer_dict[question][16]:
            correct_answers += 1
    grade = 100*correct_answers / len(answers)
    #check the grade of the student
    if grade < 60:
        passed = "NO PASS"
    else:
        passed = "PASS!"
        
    return f"{correct_answers}/{len(answers)} correct! You got {grade} grade, {passed}"

In [19]:
grade(answers, student_answers)

'1/4 correct! You got 25.0 grade, NO PASS'